# Diodenkennlinie Messung mit DAC und MCC118

## Einführung

### Was ist eine Diodenkennlinie?
Die **Diodenkennlinie** beschreibt den Zusammenhang zwischen Spannung und Strom einer Diode. Sie zeigt das charakteristische **exponentiell ansteigende Verhalten** im Durchlassbereich und den **Sperrbereich** bei negativen Spannungen.

### Ziel des Experiments
Wir messen die I-V-Charakteristik einer Diode, indem wir:
1. Mit einem **DAC** verschiedene Spannungen erzeugen
2. Mit dem **MCC118** die Diodenspannung messen  
3. Den Diodenstrom über das Ohmsche Gesetz berechnen
4. Die Kennlinie grafisch darstellen

### Physikalischer Hintergrund
Die Diodenkennlinie folgt der **Shockley-Gleichung**:

**I = I_s × (e^(qV/nkT) - 1)**

Dabei ist:
- **I_s**: Sättigungsstrom (sehr klein)
- **q**: Elementarladung  
- **V**: Diodenspannung
- **n**: Idealitätsfaktor (1-2)
- **k**: Boltzmann-Konstante
- **T**: absolute Temperatur

## Schritt 1: Bibliotheken importieren

Wir benötigen verschiedene Python-Bibliotheken für die Hardware-Steuerung und Datenanalyse:

In [ ]:
from __future__ import print_function  # Python 2/3 Kompatibilität
import spidev                          # SPI-Kommunikation mit DAC
import time                           # Wartezeiten
import lgpio                          # GPIO-Steuerung für Chip Select
import matplotlib.pyplot as plt       # Diagramme erstellen

from daqhats import mcc118, OptionFlags, HatIDs    # MCC118 Hardware-Bibliothek
from daqhats_utils import select_hat_device        # Device-Auswahl

## Schritt 2: DAC-Parameter definieren

### Was ist ein DAC?
Ein **DAC (Digital-Analog-Converter)** wandelt digitale Werte in analoge Spannungen um. Unser DAC hat eine **12-Bit Auflösung** (4096 Stufen) und kann Spannungen bis **10,5V** erzeugen.

### Hardware-Parameter:


In [1]:
# === DAC Hardware-Parameter ===
CS_PIN = 22              # GPIO Pin für Chip Select (SPI)
MAX_DAC_VALUE = 4095     # Maximum DAC-Wert (12-Bit: 2^12 - 1)
MAX_SPANNUNG = 10.5      # Maximale Ausgangsspannung in Volt

# Auflösung berechnen
SPANNUNG_PRO_BIT = MAX_SPANNUNG / MAX_DAC_VALUE  # ≈ 2.56 mV pro Bit

### Berechnung der Ausgangsspannung:
**U_out = (DAC_Wert / 4095) × 10.5V**

Beispiele:
- DAC-Wert 0 → 0V
- DAC-Wert 2048 → 5.25V  
- DAC-Wert 4095 → 10.5V

## Schritt 3: SPI-Kommunikation einrichten

### SPI-Konfiguration
Der DAC kommuniziert über die **SPI-Schnittstelle** mit speziellen Timing-Anforderungen:

In [ ]:
# === SPI-Schnittstelle konfigurieren ===
spi = spidev.SpiDev()
spi.open(0, 0)              # Bus 0, Device 0
spi.max_speed_hz = 1000000  # 1 MHz Übertragungsgeschwindigkeit
spi.mode = 0b00             # SPI Mode 0 (CPOL=0, CPHA=0)

# === GPIO für Chip Select ===
gpio_handle = lgpio.gpiochip_open(0)  # GPIO Chip 0 öffnen
if gpio_handle < 0:
    raise Exception("Fehler beim Öffnen des GPIO Chips")

# CS Pin als Ausgang konfigurieren
ret = lgpio.gpio_claim_output(gpio_handle, CS_PIN, lgpio.SET_PULL_NONE)
if ret < 0:
    raise Exception(f"Fehler beim Konfigurieren von GPIO Pin {CS_PIN}")

lgpio.gpio_write(gpio_handle, CS_PIN, 1)  # CS initial auf HIGH

### SPI-Modi Erklärung:
- **Mode 0**: CPOL=0, CPHA=0 (Clock LOW, Daten bei steigender Flanke)
- **CPOL**: Clock-Polarität (0=LOW, 1=HIGH im Ruhezustand)
- **CPHA**: Clock-Phase (0=erste Flanke, 1=zweite Flanke)

## Schritt 4: DAC-Steuerfunktion

### DAC-Datenformat
Der DAC erwartet ein **16-Bit Datenpaket** mit folgender Struktur:

| Bit | 15 | 14 | 13 | 12 | 11-0 |
|-----|----|----|----|----|------|
| **Funktion** | Channel | Buffer | Gain | Shutdown | Data |

In [ ]:
def write_dac(value):
    """
    Schreibt einen Wert an den DAC (0-4095)
    
    Parameter:
    value: DAC-Wert (0 bis 4095)
    
    DAC-Register-Aufbau (16 Bit):
    - Bit 15: Channel Select (0=A, 1=B) 
    - Bit 14: Buffer Control (1=buffered)
    - Bit 13: Gain Select (0=2x, 1=1x)
    - Bit 12: Shutdown (1=active, 0=shutdown)
    - Bit 11-0: Data (12-Bit DAC-Wert)
    """
    # Eingabe validieren
    assert 0 <= value <= MAX_DAC_VALUE, f"DAC-Wert muss zwischen 0 und {MAX_DAC_VALUE} liegen!"
    
    # === Kontrollbits zusammensetzen ===
    control = 0
    control |= 0 << 15  # Channel A auswählen (0)
    control |= 1 << 14  # Buffered Mode aktivieren (1)
    control |= 0 << 13  # Gain = 2x für vollen Spannungsbereich (0)
    control |= 1 << 12  # Normal Operation, nicht Shutdown (1)
    
    # === Datenpaket erstellen ===
    data = control | (value & 0xFFF)  # 12-Bit Daten hinzufügen
    high_byte = (data >> 8) & 0xFF    # Obere 8 Bit
    low_byte = data & 0xFF            # Untere 8 Bit

    # Debug-Ausgabe (optional)
    spannung = (value / MAX_DAC_VALUE) * MAX_SPANNUNG
    print(f"DAC: {value:4d} → {spannung:.3f}V | SPI: 0x{high_byte:02X}{low_byte:02X}")
    
    # === SPI-Übertragung ===
    lgpio.gpio_write(gpio_handle, CS_PIN, 0)  # Chip Select aktivieren
    spi.xfer2([high_byte, low_byte])          # 16-Bit Daten senden
    lgpio.gpio_write(gpio_handle, CS_PIN, 1)  # Chip Select deaktivieren

## Schritt 5: Hauptmessung durchführen
### Messablauf programmieren

In [ ]:
def cleanup_gpio():
    """
    GPIO-Cleanup mit lgpio
    """
    global gpio_handle
    
    try:
        # DAC auf 0V setzen
        write_dac(0)
    except:
        pass
    
    try:
        # GPIO freigeben und Chip schließen
        if gpio_handle is not None:
            lgpio.gpio_free(gpio_handle, CS_PIN)  # Pin freigeben
            lgpio.gpiochip_close(gpio_handle)     # Chip schließen
            gpio_handle = None
        
        # SPI schließen
        spi.close()
        
        print("GPIO-Cleanup abgeschlossen (lgpio)")
    except Exception as e:
        print(f"Cleanup-Fehler: {e}")

def main():
    """
    Hauptfunktion für die Diodenkennlinienmessung
    
    Ablauf:
    1. Messparameter vom Benutzer abfragen
    2. Hardware initialisieren
    3. Spannungsrampe fahren und dabei messen
    4. Strom aus Spannung berechnen  
    5. Kennlinien-Diagramme erstellen
    """
    print("=" * 50)
    print("    DIODENKENNLINIE MESSUNG")
    print("=" * 50)

    try:
        # === Messparameter eingeben ===
        print("\n📋 Messparameter eingeben:")
        r_serie = float(input("Serienwiderstand [Ω] (z.B. 100): "))
        anzahl_punkte = int(input("Anzahl Messpunkte (mind. 15): "))
        
        # Mindestanzahl prüfen
        if anzahl_punkte < 15:
            print("⚠️  Mindestens 15 Punkte für aussagekräftige Kennlinie!")
            anzahl_punkte = 15

        spannung_max = float(input(f"Maximale Spannung [V] (max {MAX_SPANNUNG:.1f}V): "))
        if spannung_max > MAX_SPANNUNG:
            print(f"⚠️  Begrenze auf {MAX_SPANNUNG:.1f}V.")
            spannung_max = MAX_SPANNUNG

        # === Hardware verbinden ===
        print("\n🔧 Verbinde Hardware...")
        address = select_hat_device(HatIDs.MCC_118)
        hat = mcc118(address)
        print(f"✅ MCC118 an Adresse {address} verbunden.")

        # === Messreihen vorbereiten ===
        eingestellte_spannungen = []  # U_DAC (Sollwert)
        diodenspannungen = []         # U_Diode (gemessen)
        stroeme = []                  # I_Diode (berechnet)

        print(f"\n📊 Starte Messung: 0V → {spannung_max:.1f}V in {anzahl_punkte} Schritten")
        print("=" * 80)
        print("  Nr. |  U_DAC [V] | U_Diode [V] | I_Diode [mA] | DAC-Wert")
        print("-" * 80)
        
        # === Messschleife ===
        for i in range(anzahl_punkte):
            # Spannung linear verteilen (0 bis spannung_max)
            spannung_dac = i * spannung_max / (anzahl_punkte - 1)
            
            # DAC-Wert berechnen
            dac_value = int((spannung_dac / MAX_SPANNUNG) * MAX_DAC_VALUE)
            
            # === DAC setzen ===
            write_dac(dac_value)
            time.sleep(0.1)  # Längere Einschwingzeit abwarten (erhöht von 0.05 auf 0.1)

            # === Diodenspannung messen ===
            try:
                spannung_diode = hat.a_in_read(7)  # MCC118 Kanal 7
            except Exception as e:
                print(f"Fehler bei Messung Punkt {i+1}: {e}")
                break  # Abbruch bei Fehler, um Daten bis hier zu plotten

            # === Strom berechnen ===
            strom = (spannung_dac - spannung_diode) / r_serie
            strom_ma = strom * 1000  # Umrechnung in mA für bessere Lesbarkeit

            # === Daten speichern ===
            eingestellte_spannungen.append(spannung_dac)
            diodenspannungen.append(spannung_diode)
            stroeme.append(strom)

            # === Fortschritt anzeigen ===
            print(f" {i+1:3d}. | {spannung_dac:8.3f} | {spannung_diode:9.5f} | "
                  f"{strom_ma:10.3f} | {dac_value:8d}")

        # === DAC auf 0V setzen (Sicherheit) ===
        write_dac(0)
        print("\n✅ Messung abgeschlossen!")

        # === Datenanalyse ===
        print(f"\n📈 Analysiere {len(stroeme)} Datenpunkte...")
        
        # Schwellenspannung finden (Strom > 1mA)
        schwellenspannung = None
        for u, i in zip(diodenspannungen, stroeme):
            if i > 0.001:  # 1mA Grenze
                schwellenspannung = u
                break
        
        if schwellenspannung:
            print(f"🔍 Schwellenspannung: ≈ {schwellenspannung:.3f}V")

        # === Diagramme erstellen ===
        if len(stroeme) > 0:  # Nur plotten, wenn Daten vorhanden
            print("📊 Erstelle Diagramme...")
            
            plt.figure(figsize=(14, 6))

            # === Subplot 1: Klassische I-V Kennlinie ===
            plt.subplot(1, 2, 1)
            plt.plot(diodenspannungen, [i*1000 for i in stroeme], 'ro-', 
                    linewidth=2, markersize=4, label='Messdaten')
            plt.xlabel("Diodenspannung U_D [V]", fontsize=11)
            plt.ylabel("Diodenstrom I_D [mA]", fontsize=11) 
            plt.title("Diodenkennlinie (I-V)", fontsize=12, fontweight='bold')
            plt.grid(True, alpha=0.3)
            plt.legend()
            
            # Schwellenspannung markieren
            if schwellenspannung:
                plt.axvline(schwellenspannung, color='orange', linestyle='--', 
                           label=f'Schwelle ≈ {schwellenspannung:.3f}V')
                plt.legend()

            # === Subplot 2: Eingangs- vs. Ausgangskennlinie ===
            plt.subplot(1, 2, 2)
            plt.plot(eingestellte_spannungen, [i*1000 for i in stroeme], 'bo-',
                    linewidth=2, markersize=4, label='I_D(U_DAC)')
            plt.xlabel("Eingangsspannung U_DAC [V]", fontsize=11)
            plt.ylabel("Diodenstrom I_D [mA]", fontsize=11)
            plt.title("Eingangsspannung vs. Strom", fontsize=12, fontweight='bold')
            plt.xlim(0, spannung_max)
            plt.grid(True, alpha=0.3)
            plt.legend()

            plt.tight_layout()
            plt.show()

            # === Zusammenfassung ausgeben ===
            max_strom = max(stroeme) * 1000
            max_spannung = max(diodenspannungen)
            print(f"\n📋 Messergebnisse:")
            print(f"   • Maximaler Strom: {max_strom:.2f} mA")
            print(f"   • Maximale Diodenspannung: {max_spannung:.3f} V")
            print(f"   • Serienwiderstand: {r_serie:.0f} Ω")
            print(f"   • Schwellenspannung: {schwellenspannung:.3f} V" if schwellenspannung else "   • Schwellenspannung: nicht erreicht")
        else:
            print("Keine Daten zum Plotten verfügbar.")

    except KeyboardInterrupt:
        print("\n⚠️  Messung durch Benutzer abgebrochen.")
        write_dac(0)  # Sicherheit: DAC auf 0V
    except Exception as e:
        print(f"❌ Fehler: {e}")
        write_dac(0)  # Sicherheit: DAC auf 0V
    finally:
        # === Cleanup (immer ausführen!) ===
        cleanup_gpio()

# === Messung starten ===
if __name__ == "__main__":
    main()